In [70]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps


# Import API key
from config import g_key

In [71]:
#import csv from Part 1
df = pd.read_csv("Resources/cities_data.csv")
df

,name,lat,lng,max_temp_f,humidity,cloudiness,windspeed,country,date
0,Kautokeino,69.01,23.04,44.49,87,100,25.30,NO,1600891434
1,East London,-33.02,27.91,67.75,53,12,2.42,ZA,1600891394
2,Vaini,-21.20,-175.20,71.60,94,20,9.17,TO,1600891434
3,Cape Town,-33.93,18.42,59.00,93,75,12.75,ZA,1600891146
4,Fort Nelson,58.81,-122.70,50.00,81,75,2.24,CA,1600891434
...,...,...,...,...,...,...,...,...,...
549,Kitgum,3.28,32.89,67.66,88,86,4.45,UG,1600891565
550,Chengde,40.97,117.94,56.28,88,100,1.66,CN,1600891565
551,Peñarroya-Pueblonuevo,38.30,-5.27,75.20,57,91,3.36,ES,1600891565
552,Vanimo,-2.67,141.30,76.42,87,100,3.15,PG,1600891389


In [76]:
#drop rows to find ideal weather conditions
#create rows to drop within columns column
hi_temps = df[df["max_temp_f"] > 90].index
low_temps = df[df["max_temp_f"] < 60].index
high_winds = df[df["windspeed"] > 10].index
cloudy = df[df["cloudiness"] > 40].index

df.drop(hi_temps, inplace=True)
df.drop(low_temps, inplace=True)
df.drop(high_winds, inplace=True)
df.drop(cloudy, inplace=True)
df

,name,lat,lng,max_temp_f,humidity,cloudiness,windspeed,country,date
1,East London,-33.02,27.91,67.75,53,12,2.42,ZA,1600891394
2,Vaini,-21.20,-175.20,71.60,94,20,9.17,TO,1600891434
10,São Filipe,14.90,-24.50,80.60,80,33,6.53,CV,1600891436
18,San Patricio,19.22,-104.70,89.60,66,20,6.93,MX,1600891437
25,Grand Gaube,-20.01,57.66,66.99,73,2,3.00,MU,1600891439
...,...,...,...,...,...,...,...,...,...
541,Korla,41.76,86.15,64.81,28,19,4.72,CN,1600891563
542,Nālūt,30.33,10.85,84.87,21,0,8.23,LY,1600891563
544,Bolshaya Dzhalga,45.97,42.70,62.87,60,26,9.73,RU,1600891563
546,Artyom,40.47,50.33,64.40,77,0,4.70,AZ,1600891564


In [79]:
#create lat/lng string for API calls and add to df
df["lat_lng"] = df["lat"].astype(str) + "," + df["lng"].astype(str)

#create list to iterate over
lat_lng_list = df["lat_lng"].tolist()

In [80]:

hotels = requests.get("https://maps.googleapis.com/maps/api/place/nearbysearch/json", params={
    "key": g_key,
    "location": lat_lng_list,
    "radius": 5000,
    "keyword": "hotel"
}).json()

In [89]:
hotels["results"][0]

{'business_status': 'OPERATIONAL',
 'geometry': {'location': {'lat': -33.0186916, 'lng': 27.9186969},
  'viewport': {'northeast': {'lat': -33.01733407010728,
    'lng': 27.92016887989272},
   'southwest': {'lat': -33.02003372989272, 'lng': 27.91746922010728}}},
 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png',
 'name': 'Premier Hotels & Resorts',
 'opening_hours': {'open_now': True},
 'photos': [{'height': 1335,
   'html_attributions': ['<a href="https://maps.google.com/maps/contrib/104077973295993324928">Premier Hotels &amp; Resorts</a>'],
   'photo_reference': 'CmRaAAAAX7yG-zQNUnzGv0g7yiipuoWwUdh4GpvC3Qx-S7XJnWDmGnyjfKZljUrHVk07dxyRax9ITaeVeQUJwRU-DDhL3DCGFUfgyJkN3EswFulQZBipOZc25IugWnFcPnjKHhU6EhAryLAoePqnZ4BrpBLoxDq7GhTlLHeKeikU6pKf3Lk5NvvSWSTlaA',
   'width': 2000}],
 'place_id': 'ChIJyXgrqubhZh4R4QPbGR1GRyM',
 'plus_code': {'compound_code': 'XWJ9+GF East London, South Africa',
  'global_code': '4GR9XWJ9+GF'},
 'rating': 4.2,
 'reference': 'ChIJyXgrqubhZ

In [82]:
#define function for extracting data
def extract_data(data):
    data = data["results"][0]
    return {
        "Hotel Name" : data["name"],
        "Lat" : data["geometry"]["location"]["lat"],
        "Lng" : data["geometry"]["location"]["lng"]
    } 

In [83]:
#request data from Google Places api with api calls

hotel_results = []

#set beginning print statments for output
print("Beginning Data Retrieval")
print("-" * 30)

#set up record counter for output
record_counter = 1

#create for loop to loop over random city list
for coords in lat_lng_list:
    
    #set up request and include try/except
    try:
        print(f"Processing Record {record_counter} | {coords}")
        
        record_counter += 1
        
        data = requests.get("https://maps.googleapis.com/maps/api/place/nearbysearch/json", params={
            "key": g_key,
            "location": coords,
            "radius": 5000,
            "keyword": "hotel"
        }).json()
    
        hotel_result = extract_data(data)
    
        hotel_results.append(hotel_result)
        
    except:
        print("No hotel found. Skipping ...")
        pass
    
print("-" * 30)
print("Data Retrieval Complete")
print("-" * 30)

Beginning Data Retrieval
------------------------------
Processing Record 1 | -33.02,27.91
Processing Record 2 | -21.2,-175.2
Processing Record 3 | 14.9,-24.5
Processing Record 4 | 19.22,-104.7
Processing Record 5 | -20.01,57.66
Processing Record 6 | 41.8,-73.12
Processing Record 7 | 27.67,-105.17
Processing Record 8 | 5.41,100.34
Processing Record 9 | 41.14,-73.36
Processing Record 10 | -20.44,-54.86
Processing Record 11 | 33.73,135.98
Processing Record 12 | 35.03,25.95
Processing Record 13 | -21.36,55.77
Processing Record 14 | 31.61,34.76
Processing Record 15 | -33.59,26.89
Processing Record 16 | -10.75,-77.77
Processing Record 17 | 32.67,-17.1
Processing Record 18 | 59.7,30.79
No hotel found. Skipping ...
Processing Record 19 | 16.86,-99.89
Processing Record 20 | -23.86,35.38
Processing Record 21 | 40.6,-124.16
Processing Record 22 | 38.42,-121.42
Processing Record 23 | -26.65,15.16
Processing Record 24 | 10.4,5.47
Processing Record 25 | -12.23,136.77
Processing Record 26 | -23.79,-

In [85]:
hotel_df = pd.DataFrame(hotel_results)

In [88]:
hotel_df

,Hotel Name,Lat,Lng
0,Premier Hotels & Resorts,-33.018692,27.918697
1,Paradise First Hotel,-21.155314,-175.177150
2,Colonial Guest House of Fogo,14.895686,-24.498694
3,Hotel Bogavante,19.204438,-104.684203
4,LUX Grand Gaube Resort & Villas,-20.002344,57.659766
...,...,...,...
99,Shangri-La's Le Touessrok Resort & Spa,-20.251215,57.796825
100,"дом п. Левашово, СПб",60.106090,30.189664
101,Lavande Hotel Korla Shihua Avenue,41.746445,86.168415
102,F marine resort,40.438401,50.348919
